In [19]:
import pandas as pd
import regex as re

regex = {
    "PHP":        r'(?<doc>(\/\*\*(.|\n)*?\*\/|(\/\/.*?\s*)+))?\s*(?<name>(public|protected|private)*\s*function\s+\w+)\s*(?<paren>\(([^()]+|(?&paren))*\))\s*(?<brace>\{([^{}]+|(?&brace))*\})',
    "JavaScript": r'(?<doc>(\/\*\*(?:(?!\/\*\*).|\n)*?\*\/|(\/\/.*?\s*)+))?\s*(?<name>function\s+\w+)\s*(?<paren>\(([^()]+|(?&paren))*\))\s*(?<brace>\{([^{}]+|(?&brace))*\})',
    "C":          r"(?<doc>(\/\*(?:(?!\/\*).|\n)*?\*\/|(\/\/.*?\s*)+))?\s*(?<name>(public|private|protected)?\s*(static)?\s*\w+\s+\w+)\s*(?<paren>\(([^()]+|(?&paren))*\))\s*(?<brace>\{([^{}]+|(?&brace))*\})",
    "C++":        r"(?<doc>(\/\*(?:(?!\/\*).|\n)*?\*\/|(\/\/.*?\s*)+))?\s*(?<name>(public|private|protected)?\s*(static)?\s*\w+\s+\w+)\s*(?<paren>\(([^()]+|(?&paren))*\))\s*(?<brace>\{([^{}]+|(?&brace))*\})",
    "C#":         r"(?<doc>(\/\/\/.*\s*)+)?\s*(?<name>(public|private|protected|internal|file|protected internal|private protected)?\s*(static)?\s*\w+\s+\w+)\s*(?<paren>\(([^()]+|(?&paren))*\))\s*(?<brace>\{([^{}]+|(?&brace))*\})",
    "Java":       r"(?<doc>(\/\*\*(?:(?!\/\*\*).|\n)*?\*\/|(\/\/.*?\s*)+))?\s*(?<name>(public|private|protected)?\s*(static)?\s*\w+\s+\w+)\s*(?<paren>\(([^()]+|(?&paren))*\))\s*(?<brace>\{([^{}]+|(?&brace))*\})",
    "TypeScript": r'(?<doc>(\/\*\*(?:(?!\/\*\*).|\n)*?\*\/|(\/\/.*?\s*)+))?\s*(?<name>function\s+\w+)\s*(?<paren>\(([^()]+|(?&paren))*\))\s*(?<brace>\{([^{}]+|(?&brace))*\})',
    "Shell":      r"(?<doc>(\#.*?\s*)+)?\s*(?<name>\w+)\s*(?<paren>\(([^()]+|(?&paren))*\))\s*(?<brace>\{([^{}]+|(?&brace))*\})",
    "Ruby":       r'(?<doc>(\#.*?\s*)+)?\s*(?<name>def\s+\w+)\s*(?<paren>\(([^()]+|(?&paren))*\))\s*(?<brace>\{([^{}]+|(?&brace))*\})',
    "Python": r""
}

keywords = ["if", "while", "do", "else", "switch", "elseif", "elif", "match", "for", "try", "catch"]

In [20]:
import regex as re

def add_curly_brace(match):
    return match.group(1) + " {"

def Ruby_hotfix(col):
    col = ruby_regex.sub(add_curly_brace, col)
    col = ' '.join(word_map.get(word, word) for word in col.split())
    return col


def reverse_Ruby_hotfix(functions):
    if functions:
        for func in functions:
            func[-1] = ' '.join(reverse_word_map.get(word, word) for word in func[-1].split())
            func[2] = ' '.join(reverse_word_map.get(word, word) for word in func[2].split())
            
            i = func[-1].find('{')
            if i != -1:
                func[-1] = func[-1][:i] + func[-1][i + 1:]
            if func[2][0] == "{":
                func[2] = func[2][1:]
            
    return functions


word_map = { 
    "do":"do{", 
    "if":"if{", 
    "unless":"unless{", 
    "case":"case{", 
    "while":"while{", 
    "until":"until{",
    "end":"end}",
    "class":"class{",
    "module":"module{"
}

reverse_word_map = {v: k for k, v in word_map.items()}

ruby_regex = re.compile(r'(def\s+\w+\s*(?<paren>\(([^()]+|(?&paren))*\)))')

In [ ]:
whitespace = re.compile(r'\s+')
def extract_functions(code):
    
    functions = []
    matches = re.finditer(pattern, code)
        
    for match in matches:
        docstring = match.group("doc")
        name = match.group("name")
        parameters = match.group("paren")
        body = match.group("brace").strip()
        functions.append([name, parameters, body, docstring])
        
    return functions

In [22]:
def filter_function(example):
    return example['max_stars_count'] and example['max_stars_count'] > 2 and example['lang'] in regex.keys() and example['size'] < 20000

In [23]:
def handle_data(example, df):
    functions_data = []
    if filter_function(example):
        lang = example['lang']
        
        if lang == "Ruby":
            example['content'] = Ruby_hotfix(example['content'])
        
        functions = extract_functions(example['content'])
        
        if language == "Ruby":
            functions = reverse_Ruby_hotfix(functions)
        
        if functions:
            for function in functions:
                if function:
                    if re.sub(whitespace, '', function[0]) in keywords: continue
                    # if len(function[2]) > 1000: continue
                    if not function[2].replace('{','').replace('}','').replace(':','').replace('\n','').strip(): continue
                    if not function[3] or (function[3] and not function[3].replace("//","").replace("///","").replace("#","").replace("//*","").replace("*/","").replace("'''","").replace('"""',"").strip()): continue
                    
                    functions_data.append({
                        'name':function[0],
                        'params':function[1],
                        'body':function[2],
                        'docstring':function[3],
                        #'full_code':function[4],
                        'file_id':example['hexsha'],
                        'language':lang,
                    })
    return functions_data        

In [ ]:
%%time
import pandas as pd
import ast

def extract_python_functions(code, error_counter, hexsha):
    functions = []
    try:
        code = code.encode('utf-8', 'ignore').decode('utf-8', 'ignore')
        tree = ast.parse(code)
        for node in ast.walk(tree):
            if isinstance(node, ast.FunctionDef):
                function_name = node.name.encode('utf-8', 'ignore').decode('utf-8', 'ignore')
                params = str([arg.arg.encode('utf-8', 'ignore').decode('utf-8', 'ignore') for arg in node.args.args])
                function_code = ast.get_source_segment(code, node).encode('utf-8', 'ignore').decode('utf-8', 'ignore')
                docstring = ast.get_docstring(node)
                if docstring: docstring = docstring.encode('utf-8', 'ignore').decode('utf-8', 'ignore')
                body = "\n".join([ast.unparse(stmt) for stmt in node.body]).encode('utf-8', 'ignore').decode('utf-8', 'ignore')
                if hexsha: hexsha = hexsha.encode('utf-8', 'ignore').decode('utf-8', 'ignore')
                
                functions.append([
                    function_name,
                    params,
                    body,
                    docstring,
                    function_code,
                    hexsha
                    ])
    except (SyntaxError, ValueError, UnicodeEncodeError, UnicodeDecodeError):
        error_counter += 1
    return functions, error_counter

        
def handle_py_data(example, df, py_error_count):
    functions_data = []
    if filter_function(example):
        #print(example, flush=True)
        hexsha = example['hexsha']
        functions, py_error_count = extract_python_functions(example['content'], py_error_count, hexsha)

        if functions:
            for function in functions:
                if function:
                    functions_data.append({
                        'name':function[0],
                        'params':function[1],
                        'body':function[2],
                        'docstring':function[3],
                        #'full_code':function[4],
                        'file_id': function[5],
                        'language':'Python',
                    })
    return functions_data, py_error_count

CPU times: total: 0 ns
Wall time: 0 ns


In [ ]:
from huggingface_hub import login
login(token="XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\User\.cache\huggingface\token
Login successful


In [26]:
def list_of_dicts_to_dict_of_lists(data_list):
    dict_of_lists = {}
    for d in data_list:
        for key, value in d.items():
            if key not in dict_of_lists:
                dict_of_lists[key] = []
            dict_of_lists[key].append(value)
    return dict_of_lists

In [ ]:
%%time 

# 544 minutes +

import json
import time
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from datasets import load_dataset, Dataset
from huggingface_hub import HfApi, HfFolder

chunk_size = 1000000
hf_repo_id = "JanDkff/TinyFuncData-new"
api = HfApi()
token = HfFolder.get_token()

#expected_features = Features({
#    'name': Value(dtype='string'),
#    'params': Value(dtype='string'),
#    'body': Value(dtype='string'),
#    'docstring': Value(dtype='string'),
#    'file_id': Value(dtype='string'),
#    'language': Value(dtype='string')
#})


processed_data = []

for language in regex.keys():
    i = 0
    j = 0
    start_time = time.time()
    dataset = load_dataset('bigcode/the-stack-dedup', data_dir=f'data/{language.lower().replace("++","pp").replace("#","-sharp")}', streaming=True, split="train")
    pattern = regex[language]
    for example in dataset:
        if not i % 50000:
            current_time = time.time()
            print(i, language, round(current_time - start_time, 2), flush=True)
            if processed_data: print(processed_data[-1])
            #start_time = current_time
        i += 1

        processed_data += handle_data(example, dataset)
        if len(processed_data) >= chunk_size:
            temp_dataset = Dataset.from_dict(list_of_dicts_to_dict_of_lists(processed_data))
            temp_dataset.push_to_hub(hf_repo_id, token=token, split=f'part_{j}_{language.lower().replace("++","pp").replace("#","sharp")}')
            print(j)
            j += 1
            processed_data = []
            temp_dataset = None

    if processed_data:
        temp_dataset = Dataset.from_dict(list_of_dicts_to_dict_of_lists(processed_data))
        temp_dataset.push_to_hub(hf_repo_id, token=token, split=f'part_{j}_{language.lower().replace("++","pp").replace("#","sharp")}')
        processed_data = []
        temp_dataset = None


0 PHP 6.61


KeyboardInterrupt: 

In [ ]:
%%%%%%%%%%%%%%%%%%%%%%%%

In [118]:
len(processed_data)

928043

In [66]:
%%time
import json
import time
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from datasets import load_dataset

# Define the chunk size
chunk_size = 1000
py_error_count = 0

# Process the dataset in chunks
i = 0

start_time = time.time()
processed_data = []

# Initialize ParquetWriter
parquet_file = 'new_dataset.parquet'
schema = None
writer = None

try:
    language = 'Python'
    for example in dataset:
        i += 1
        
        if not i % 50000:
            current_time = time.time()
            print(i, language, round(current_time - start_time, 2), flush=True)
            start_time = current_time
        
        data, py_error_count = handle_py_data(example, dataset, py_error_count)
        processed_data += data
        if len(processed_data) >= chunk_size:
            # Convert the processed data to a DataFrame
            df = pd.DataFrame(processed_data)
            table = pa.Table.from_pandas(df)

            if schema is None:
                schema = table.schema
                writer = pq.ParquetWriter(parquet_file, schema)

            # Write the Table to the Parquet file
            writer.write_table(table)

            # Clear the list to free memory
            processed_data.clear()

    # Write any remaining data
    if processed_data:
        df = pd.DataFrame(processed_data)
        table = pa.Table.from_pandas(df)
        if writer is None:
            writer = pq.ParquetWriter(parquet_file, table.schema)
        writer.write_table(table)

finally:
    # Close the ParquetWriter
    if writer:
        writer.close()


<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal


50000 Python 112.42


<unknown>:24: SyntaxWarning: invalid decimal literal


100000 Python 113.46
150000 Python 111.03
200000 Python 107.22


<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:343: SyntaxWarning: invalid decimal literal


...

11950000 Python 131.14
12000000 Python 134.52
12050000 Python 118.57
12100000 Python 141.5
12150000 Python 115.03
12200000 Python 145.17
12250000 Python 115.84
12300000 Python 139.55
12350000 Python 122.93
12400000 Python 128.25
12450000 Python 136.41
12500000 Python 111.21
12550000 Python 154.91
12600000 Python 117.98
12650000 Python 141.88
12700000 Python 122.62


<unknown>:22: SyntaxWarning: invalid decimal literal


12750000 Python 139.27
12800000 Python 134.4


<unknown>:1: SyntaxWarning: invalid decimal literal


12850000 Python 140.92
12900000 Python 144.16
12950000 Python 143.68
CPU times: total: 7h 29min 58s
Wall time: 8h 59min 4s


In [28]:
%%time
import json
import time
from datasets import load_dataset


# Define the chunk size
chunk_size = 1000
py_error_count = 0

# Process the dataset in chunks
i = 0

start_time = time.time()
processed_data = []

for language in ["Shell","Python"]:
    dataset = load_dataset('bigcode/the-stack-dedup', data_dir=f'data/{language.lower().replace("#","-sharp").replace("++","pp")}', streaming=True, split="train")
    pattern = regex[language]
    for example in dataset:
        if not i%50000:
            current_time = time.time()
            print(i, language, round(current_time - start_time, 2), flush=True)
            start_time = current_time
        i += 1

        processed_data += handle_data(example, dataset)
        if len(processed_data) >= chunk_size:
            # Save the data to a file
            with open('new_dataset.json', 'a') as f:
                for item in processed_data:
                    f.write(json.dumps(item) + '\n')

            # Clear the list to free memory
            processed_data.clear()

    # Save any remaining data
    if processed_data:
        with open('new_dataset.json', 'a') as f:
            for item in processed_data:
                f.write(json.dumps(item) + '\n')

KeyboardInterrupt: 

In [ ]:
['Python' 'PHP' 'JavaScript' 'C' 'C++' 'Ruby']
55598072

In [65]:
temp_dataset

Dataset({
    features: ['name', 'params', 'body', 'docstring', 'file_id', 'language'],
    num_rows: 2500003
})

In [67]:
            temp_dataset = Dataset.from_dict(list_of_dicts_to_dict_of_lists(processed_data))
            temp_dataset.push_to_hub(hf_repo_id, token=token, split=f"part_{j}")
            print(j)
            j += 1

Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1251 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1251 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/5.44k [00:00<?, ?B/s]

30


In [69]:
    processed_data = []
    temp_dataset = None

In [73]:
j

37